# Finishing mri preprocessing

After it's been run through fmriprep, we perform a few additional preprocessing steps. This might get tinkered with as we perfect our processes, and it might not be appropriate for all analyses (we'll need slightly different steps for MVPA, for example). 

All of this code can be edited (or run) in the terminal by accessing the script B3/scripts/step_two_afterfmriprep.sh. For participants who only had one run of the first tasks, you'll instead want to run step_2_afterfmriprep_1run.sh. 

### Prerequisite steps: 
By the time you get here, you should have:
* Mapped the DOP-Restricted drive to your computer.
* Run the initial preprocessing for a subject (STEP_ONE_Preprocessing.ipynb or step_one_prepreocessingin.sh).
* Navigated to your home directory (/home/YOUR_PEDS_ID@pediatrics.wisc.edu) and confirmed that preprocessing is complete (you should see a directory AND a .html file for each subject.  

## Step 1: Moving the files out of your home directory

First, we need to move the files out of our home directory and into the fmriprep-outputs directory in B3. This can be done by running the code below or by running this command directly in the terminal. 

You'll need to edit the code below to include the right subject IDs and lsyour Peds ID.
#### Note: 
If you run these commands in the terminal, you will see a while bunch of "operation not permitted" messages. I am about 99.8% sure it is safe to ignore these. 

In [ ]:
%%bash
pedsid="YOUR_PEDS_ID"
for subj in 008; do 
mv sub-${subj}* ../mnt/DOP-Restricted_Share/B3_Moreno-Selkie-Cascio_2022-1280/B3_Project_2_Cascio/B3/fmriprep_outputs/
done

## Step 2: Checking data quality

Before going any farther, you'll want to run a quick check to confirm the quality of the preprocessed data. We do a lot of data manipulation in fMRI analysis, so it's important to check our data often to catch mistakes early. 

In the file explorer on your computer, navigate to the fmriprep_outputs directory (full path in the code above) and open the file sub-${subj}.html.

In a separate window, navigate to B3/QA_checks and open the file "QA checklist.docx"

Complete the form as best you can, and save it as sub-${subj}.docx when you're finished. If you notice major problems in this process (e.g. big black streaks throughout the brain, missing chunks of the brain, or other serious distortions), save the report in "Needs review" (distortions in the carpet plot are OK) and move the subject directory and .html to B3/need_review. Otherwise, save the report in the "Clean" or "OK mostly" directory, depending on your judgment. 

#### Note:
This is one of those processes that will feel confusing at first, but after reviewing a few brains you should soon understand what to expect from a normal, "clean" dataset. This will become second nature, I promise. 

## Step 3: Trimming, smoothing, and scaling

Next, we perform a few additional preprocessing steps. Steps 2 and 3 use code that is explained in [Andy's Brain Book](https://andysbrainbook.readthedocs.io/en/latest/OpenScience/OS/fMRIPrep_Demo_4_AdditionalPreproc.html).

What this code does is:
1. Remove the first 7 TRs of all functional runs. We do this because the signal has not yet reached a "steady state" 
2. Smooth or "blur" the data so that signal in each voxel is recoded as a weighted average of its signal and the signal in the surrounding voxels. This is a practice that's meant to reduce noise. 
3. Scale the data so that the signal can be interpreted as percent signal change. 

To run the script, you'll just need to input the correct subject IDs. The script will take some time to run --- when it finishes, you should see "trimmed_bold", "blur", "mean" and "scale" files in the subjects's "func" directory.

Expect to see a bunch of errors regarding run 02 of cyberball. This is just because I built the loops a bit sloppily, so the script is looking for a run that doesn't exist. They can be ignored. 

In [1]:
%%bash

for subj in 680; do

cd ../fmriprep_outputs/sub-${subj}/ses-01/func

   for run in 01 02; do

     for task in conformity cyberball peerfeedback selfother; do
     #Remove first 7 TRs to achieve steady-state
     3dTcat -prefix sub-${subj}_ses-01_task-${task}_run-${run}_trimmed_bold.nii \
     sub-${subj}_ses-01_task-${task}_run-${run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'[7..$]'

     #Apply 4mm gaussian blur kernel <NUMBER MAY NOT BE OPTIMAL>
     3dmerge -1blur_fwhm 4.0 -doall -prefix sub-${subj}_ses-01_task-${task}_run-${run}_blur.nii \
     sub-${subj}_ses-01_task-${task}_run-${run}_trimmed_bold.nii

     #Scale values to represent percent signal change
     3dTstat -prefix sub-${subj}_ses-01_task-${task}_run-${run}_mean.nii \
     sub-${subj}_ses-01_task-${task}_run-${run}_blur.nii

     3dcalc -a sub-${subj}_ses-01_task-${task}_run-${run}_blur.nii -b sub-${subj}_ses-01_task-${task}_run-${run}_mean.nii \
     -c sub-${subj}_ses-01_task-${task}_run-${run}_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz \
     -expr 'c*min(200,a/b*100)*step(a)*step(b)' \
     -prefix sub-${subj}_ses-01_task-${task}_run-${run}_scale.nii

     done
   done

cd ../../../../scripts
done


Program 3dmerge 
3dmerge: edit and combine 3D datasets, by RW Cox
++ editing input dataset in memory (561.1 MB)
..............................................................................................................................................................................................................................................................................

Program 3dmerge 
3dmerge: edit and combine 3D datasets, by RW Cox
++ editing input dataset in memory (1037.0 MB)
...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Pr

++ 3dTcat: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ elapsed time = 9.9 s
++ 3dmerge: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ default -1dindex = 0
++ default -1tindex = 1
-- Wrote edited dataset: ./sub-680_ses-01_task-conformity_run-01_blur.nii
++ 3dTstat: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ Authored by: KR Hammett & RW Cox
++ Output dataset ./sub-680_ses-01_task-conformity_run-01_mean.nii
++ 3dcalc: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset ./sub-680_ses-01_task-conformity_run-01_scale.nii
++ 3dTcat: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ elapsed time = 17.1 s
++ 3dmerge: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ default -1dindex = 0
++ default -1tindex = 1
-- Wrote edited dataset: ./sub-680_ses-01_task-cyberball_run-01_blur.nii
++ 3dTstat: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ Authored by: KR Hammett & RW Cox
++ Output dataset ./sub-680_ses-01_task-c

## Step 4: Pulling and formatting motion confounds

When we do our first-level analyses, we'll need to control for any signal changes that can be explained by small head movements. Thankfully, fmriprep measures the motion of each participant's head at each second of the process. 

This script:
1. Pulls the six major motion regressors out of a larger matrix of confound regressors
2. Estimates the timing of each run and adds a string of zeroes to the beginning or end

To run the script, you'll just need to edit the subject ID. You'll know it worked because you'll see "rot" and "trans" files in the subject's "func" directory.

Again, expect to see errors re: run 2 of cyberball. These can be ignored. 

#### Note: 
The six confound regressors this creates for each run are considered the bare minimum for fMRI analysis. At some point in the future, we will almost certainly need to add a bunch more. 

In [2]:
%%bash

for subj in 680; do
  dir=../fmriprep_outputs/sub-${subj}/ses-01/func
        for task in conformity cyberball peerfeedback selfother;do
                rm $dir/*zeros_tmp.txt
                NT1=`3dinfo -nt $dir/sub-${subj}_ses-01_task-${task}_run-01_scale.nii`
                NT2=`3dinfo -nt $dir/sub-${subj}_ses-01_task-${task}_run-02_scale.nii`
                for ((i=0; i<$NT1; i++)); do
                        echo 0 >> $dir/r1_zeros_tmp.txt;
                done
                for ((i=0; i<$NT2; i++)); do
                        echo 0 >> $dir/r2_zeros_tmp.txt;
                done
                for reg in trans_x trans_y trans_z rot_x rot_y rot_z; do
                        for run in 01 02; do
                                awk -v col=$reg 'NR==1{for(i=1;i<=NF;i++){if($i==col){c=i;break}} print $c} NR>1{print $c}' \
                                $dir/sub-${subj}_ses-01_task-${task}_run-${run}_desc-confounds_timeseries.tsv > $dir/sub-${subj}_ses-01_task-${task}_run-${run}_${reg}_all_tmp.txt
                                sed '1d' $dir/sub-${subj}_ses-01_task-${task}_run-${run}_${reg}_all_tmp.txt > $dir/sub-${subj}_ses-01_task-${task}_run-${run}_${reg}_noHead_tmp.txt
                                sed '1!d' $dir/sub-${subj}_ses-01_task-${task}_run-${run}_${reg}_all_tmp.txt > $dir/sub-${subj}_ses-01_task-${task}_run-${run}_${reg}_Head_tmp.txt
                                tail -n +8 $dir/sub-${subj}_ses-01_task-${task}_run-${run}_${reg}_noHead_tmp.txt > temp.txt && mv temp.txt $dir/sub-${subj}_ses-01_task-${task}_run-${run}_${reg}_noHead_tmp.txt
                        done
                        cat $dir/sub-${subj}_ses-01_task-${task}_run-01_${reg}_noHead_tmp.txt $dir/r2_zeros_tmp.txt  > $dir/sub-${subj}_ses-01_task-${task}_run-01_${reg}.txt
                        cat $dir/r1_zeros_tmp.txt $dir/sub-${subj}_ses-01_task-${task}_run-02_${reg}_noHead_tmp.txt > $dir/sub-${subj}_ses-01_task-${task}_run-02_${reg}.txt

                done
        done

   cd ../fmriprep_outputs/sub-${subj}/ses-01/func
    for reg in trans_x trans_y trans_z rot_x rot_y rot_z; do
      for run in 01 02; do
        awk -v cold=$reg 'NR==1{for(i=1;1<=NF;i++){if($i==col){c=1;break}} print $c} NO >1{print $c}' \
        sub-${subj}_ses-01_task-selfother_run-${run}_desc-confounds_timeseries.tsv > ${reg}_run${run}_all_tmp.txt;
          sed '1d' ${reg}_run${run}_all_tmp.txt > ${reg}_run${run}_noHead_tmp.txt
          sed '1!d' ${reg}_run${run}_all_tmp.txt > ${reg}_run${run}_Head_tmp.txt

done
done
   cd ../../../../scripts
done



awk: cannot open ../fmriprep_outputs/sub-680/ses-01/func/sub-680_ses-01_task-cyberball_run-02_desc-confounds_timeseries.tsv (No such file or directory)
cat: ../fmriprep_outputs/sub-680/ses-01/func/r2_zeros_tmp.txt: No such file or directory
awk: cannot open ../fmriprep_outputs/sub-680/ses-01/func/sub-680_ses-01_task-cyberball_run-02_desc-confounds_timeseries.tsv (No such file or directory)
cat: ../fmriprep_outputs/sub-680/ses-01/func/r2_zeros_tmp.txt: No such file or directory
awk: cannot open ../fmriprep_outputs/sub-680/ses-01/func/sub-680_ses-01_task-cyberball_run-02_desc-confounds_timeseries.tsv (No such file or directory)
cat: ../fmriprep_outputs/sub-680/ses-01/func/r2_zeros_tmp.txt: No such file or directory
awk: cannot open ../fmriprep_outputs/sub-680/ses-01/func/sub-680_ses-01_task-cyberball_run-02_desc-confounds_timeseries.tsv (No such file or directory)
cat: ../fmriprep_outputs/sub-680/ses-01/func/r2_zeros_tmp.txt: No such file or directory
awk: cannot open ../fmriprep_outputs

## That's it!

Congrats! You've just finished preprocessing. As a next step, check out the STEP_THREE_First_Level_Analysis notebook in this directory.

In the future, you can run all of these scripts with the command step_two_afterfmriprep.sh in this directory. As with the other scripts, you'll need to edit the script slightly to add the correct subject IDs. 

#### Note: 
This process is a little bit different for participants who only have one run! For those participants, run step_two_afterfrmiprep_1run.sh.

Thanks and cheers, 

Matt